In [76]:
import random
from urllib.parse import urlparse, parse_qs
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import re
from matplotlib import pyplot as plt
import time
import urllib
import spacy
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import string
import pandas as pd
import http

#Lib require for google drive
from io import BytesIO
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from pydrive2.files import GoogleDriveFile
import mimetypes
import magic

In [159]:
user_agents = ['Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/37.0.2062.94 Chrome/37.0.2062.94 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/600.8.9 (KHTML, like Gecko) Version/8.0.8 Safari/600.8.9',
 'Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4',
 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10240',
 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/600.7.12 (KHTML, like Gecko) Version/8.0.7 Safari/600.7.12',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.8.9 (KHTML, like Gecko) Version/7.1.8 Safari/537.85.17',
 'Mozilla/5.0 (iPad; CPU OS 8_4 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H143 Safari/600.1.4',
 'Mozilla/5.0 (iPad; CPU OS 8_3 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12F69 Safari/600.1.4',
 'Mozilla/5.0 (Windows NT 6.1; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; Touch; rv:11.0) like Gecko',
 'Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/600.6.3 (KHTML, like Gecko) Version/8.0.6 Safari/600.6.3',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/600.5.17 (KHTML, like Gecko) Version/8.0.5 Safari/600.5.17',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
 'Mozilla/5.0 (iPhone; CPU iPhone OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4',
 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (iPad; CPU OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53',
 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
 'Mozilla/5.0 (X11; CrOS x86_64 7077.134.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.156 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.7.12 (KHTML, like Gecko) Version/7.1.7 Safari/537.85.16',
 'Mozilla/5.0 (Windows NT 6.0; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (iPad; CPU OS 8_1_3 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12B466 Safari/600.1.4',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/600.3.18 (KHTML, like Gecko) Version/8.0.3 Safari/600.3.18',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36']


In [4]:
def construct_url_params(url, kwargs):
    """
        Construct new url by adding query params to the url.

        Returns: new url
    """

    #if no query params add ? else add & at the end of url
    url+='?' if not '?' in url else '&'
    for i,j in kwargs.items():
        url+=f'{i}={j}&'

    #strip extra & at the end
    return url.rstrip('&')

In [58]:
class GoogleDriveWrapper:
    def __init__(self):
        auth = GoogleAuth()

        # Try to load saved client credentials
        auth.LoadCredentialsFile("creds.txt")
        if auth.credentials is None:
            # Authenticate if they're not there
            auth.GetFlow()
            auth.flow.params.update({'access_type': 'offline'})
            auth.flow.params.update({'approval_prompt': 'force'})
            auth_url = auth.GetAuthUrl()

            auth.LocalWebserverAuth()
        elif auth.access_token_expired:
            # Refresh them if expired
            auth.Refresh()
        else:
            # Initialize the saved creds
            auth.Authorize()
        # Save the current credentials to a file
        auth.SaveCredentialsFile("creds.txt")

        self.drive = GoogleDrive(auth)
        
    def getFileID(self, file_name, parent_id=None, url=False):
        """
            Function to get file id by name, if file is not present, return None
        """
        query = f"mimeType != 'application/vnd.google-apps.folder' and title='{file_name}'"
        if parent_id:
            query+= f" and '{parent_id}' in parents"

        file_list = self.drive.ListFile({'q':query}).GetList()
        for file in file_list:
            if url:
                return file['alternateLink']
            return file['id']
        
        return None
    
    
    def getFolderID(self, folder_name, parent_id=None, url=False):
        """
            Function to get folder id by name, if folder is not present, return None
        """
        
        #create the query to list the folder
        query = f"mimeType = 'application/vnd.google-apps.folder' and title='{folder_name}'"
        
        if parent_id:
            query+= f" and '{parent_id}' in parents"

        file_list = self.drive.ListFile({'q':query}).GetList()
        for file in file_list:
            if url:
                return file['alternateLink']
            return file['id']
        
        return None
    
    
    def createFolder(self, folder_name, parent_id=None):
        """
            Function to create a folder under a folder (if parent_id is present), return folder_id
        """
        #check if folder exists
        folder_id = self.getFolderID(folder_name, parent_id)
        
        if folder_id:
            return folder_id
        
        # folder not present, create the folder
        folder_metadata = {
            'title': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
        }
        
        if parent_id:
            folder_metadata['parents'] = [{'id': parent_id}]
        
        folder = self.drive.CreateFile(folder_metadata)
        folder.Upload()     
        
        return folder['id']
    
    
    
    def uploadFile(self, file_name, file_path=None, file_url=None, parent_id=None):
        """
            Function to upload File, 
            
            file_name: Name of the file to upload
            file_path: Upload the file on file path
            file_url: Upload a file from a url
            parent_id: Parent Folder Id
            
            Either Filepath or File_url is required to upload, if both present, picks file_path over file_url
        """
    
        # file metadata to upload
        file_metadata = {
            'title': file_name,
            'parents': [{'id': parent_id}] if parent_id else []
        }

        #upload a file from file_path
        if file_path:
            #get the mime_type of file
            file_metadata['mime_type'] = mimetypes.guess_type(file_path)[0]
            
            #create the file and set the content of file
            media = self.drive.CreateFile(metadata=file_metadata)
            media.SetContentFile(filename=file_path)

        elif file_url:
            
            # Make request to load the file from url
            headers = {'User-Agent': random.choice(user_agents)}
            req = urllib.request.Request(file_url, headers=headers)

            #if file size is greater, file loads partially
            max_retries = 100
            retry_count = 0
            remote_file = b""

            while retry_count <= max_retries:
                try:
                    #request to load the file
                    response = urllib.request.urlopen(req)
                    
                    #if request is succefull, read the file response and break the loop
                    remote_file = response.read()
                    break
                except http.client.IncompleteRead as e:
                    #if incompleteread exception, load the file partially 
                    remote_file += e.partial
                    retry_count += 1
                    print(f"IncompleteRead error. Retrying... (attempt {retry_count}/{max_retries})")
                    
                except urllib.error.HTTPError as err:
                    if err.status == 429:
                        #exponential backoff time
                        delay = min(5*2**retry_count, 40)
                        if delay > 40:
                            break
                            
                        
                        print(f"HTTP Error 429: sleeping for {delay} seconds!!")
                        
                        time.sleep(delay)
                        retry_count += 1
                    else:
                        print(err)
                        break
                except Exception as err:
                    print(err)
                    break

            #if no content, return None
            if not remote_file:
                return None


            #get the mime type of a file
            file_metadata['mime_type'] = mimetypes.guess_type(file_name)[0]
            
            #create the file and convet the file to BytesIO since drive requires to read file in string where it encodes the file
            media = self.drive.CreateFile(metadata=file_metadata)
            media.content = BytesIO(remote_file)

        else:
            raise Exception("Required either file_path or file_url")

        #uplaod the file
        media.Upload()

        print(f"File '{file_name}' has been uploaded to Google Drive.")

        #return the web link
        return media['alternateLink']

def guessMIMEType(file_path = None, memory_file=None):
    try:
        #return the mimetype for filepath
        if file_path:
            return mimetypes.guess_type(file_path)[0]
        
        #find the mimetype for file stored in memory
        elif memory_file:
            mime = magic.Magic(mime=True)
            mime_type = mime.from_buffer(memory_file.read())

            memory_file.seek(0)  # Reset the file pointer after reading the content

            return mime_type
        else:
            raise Exception("Required either file_path or memory_file")
    except Exception as err:
        print(err)
        return 'application/octet-stream'

In [5]:
class WalmartScraper:
    base_url = "https://www.walmart.com/search?q="

    def __init__(self, product_name, chrome_path=r"C:\Users\Priyanshi Singh\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"):

        service = webdriver.chrome.service.Service()#executable_path=chrome_path)
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument(f'--user-agent={random.choice(user_agents)}')
        options.add_argument("--disable-blink-features=AutomationControlled") 
        # Exclude the collection of enable-automation switches 
        options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
        options.add_experimental_option("detach", False) 
        
        # Turn-off userAutomationExtension 
        options.add_experimental_option("useAutomationExtension", False)
        self.driver = webdriver.Chrome(service=service, options=options)
        self.search_url = self.base_url+ re.sub("\s+", "+", product_name)
        self.driver.get(self.search_url)


    def getProductURLList(self):
        # Wait for the pagination element to be present
        WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[@id='results-container']")))

        total_pages = int(self.driver.find_elements(By.XPATH, "//nav[@aria-label='pagination']//li[not(@area-hidden) and div]")[-1].text)
        product_urls = []

        # Loop through each page
        for curr_page in range(1, total_pages + 1):
            # Get the product URLs on the current page
            products = self.driver.find_elements(By.XPATH, "//div[@data-item-id]//a")

            for product in products:
                product_urls.append(product.get_attribute("href"))

            # Go to the next page if not on the last page
            if curr_page < total_pages:
                next_page_url = construct_url_params(self.search_url, {'page': curr_page + 1})
                self.driver.get(next_page_url)
        return product_urls

    def getProductNameAndIDFromURL(self):
        product_path = urlparse(self.driver.current_url).path.strip('/').split('/')
        product_name = product_path[-2].replace("-", " ")
        product_id = product_path[-1]
        return product_name, product_id
   
    def getLeftImageUrls(self):
        # Find all image elements within the left panel
        thumbnails = self.driver.find_elements(By.XPATH, "//div[@data-testid='vertical-carousel-container']//img")
        # Extract and return the 'src' attribute from each image
        image_urls = [thumbnail.get_attribute('src') for thumbnail in thumbnails]
        # Parse each image URL using urlparse and create a new list of formatted URLs
        formatted_urls = [f"{url_res.scheme}://{url_res.netloc}/{url_res.path}" for image_url in image_urls for url_res in map(urlparse, [image_url])]
        return formatted_urls

    def parseWalmartProduct(self):
        product_detail = {}
    
        # Brand Name
        brand_element = self.driver.find_element(By.XPATH, "//section[@data-pcss-show='true']//a")
        product_detail['product_brand'] = brand_element.text
    
        # Product Name
        product_name_element = self.driver.find_element(By.XPATH, "//h1[@id = 'main-title']")
        product_detail['product_title'] = product_name_element.text
    
        # Rating Number
        rating_element = self.driver.find_elements(By.XPATH, "//div[@data-testid='reviews-and-ratings']//span[contains(@class, 'rating-number')]")
        
        product_detail['rating_number'] = rating_element[0].text.replace('(','').replace(')', '') if rating_element else ''

        price_element = self.driver.find_elements(By.XPATH, "//span[@itemprop='price']")
        product_detail['price'] = price_element[0].text if price_element else ''

        return product_detail

    def parseWalmartProductDetails(self):
        walmart_product_detail = {}          
        # Parse product details
        product_details_section = self.driver.find_elements(By.XPATH, "//section[@aria-describedby]") #//div[@data-testid='product-description-content']")
        
        for i, section in enumerate(product_details_section):
            headers = section.find_elements(By.XPATH, ".//div[contains(@class, 'expand-collapse-header')]")
            header =  headers[0].text.strip() if headers else None
            if section.find_elements(By.XPATH, ".//div[@data-testid='product-description-content']"):
                #product description div
                divs = section.find_elements(By.XPATH, ".//div[@class='mb3']")
        
                walmart_product_detail['short_description'] = divs[0].text.strip()
        
                long_description = ''
                for div in divs[1:]:
                    inner_div  = div.find_elements(By.XPATH, ".//div[@class='dangerous-html mb3']")
        
                    inner_div_header = None
                    if inner_div:
                        #next tag is bold
                        inner_div_header = inner_div[0].find_elements(By.XPATH, "./b")
        
                
                    inner_div_header = inner_div_header[0].text.rstrip(':') if inner_div_header else None
                    if inner_div_header == 'Key Features':
                        #get all divs with class dangerous-html mb3, get immediate next div, get all divs under it which has some inner TAG
                        feature_divs = div.find_elements(By.XPATH, ".//div[@class='dangerous-html mb3']/div//div[./*]")
        
                        for feature_div in feature_divs:
                            if feature_div.text.strip(':')=='Specifications':
                                #now get all the next div tags till the div tag does not have any innner tag
                                for spec in feature_div.find_elements(By.XPATH, "following-sibling::div[not(div)]"):
                                    features =  spec.text.strip().split(':')
                                    if features:
                                        walmart_product_detail[re.sub('^\d+\.', '', features[0])] = features[1]
        
                                    else:
                                        #add to long description
                                        long_description += '\n' + spec.text.strip()
        
                            else:
                                long_description += '\n' + feature_div.text.strip()
                    else:
                        
                        long_description += "\n" + div.text.strip()
        
                walmart_product_detail['long_description'] = long_description
            elif header == "Specifications":
                div_contents = section.find_elements(By.XPATH, ".//div[contains(@class, 'expand-collapse-content')]//div[@class='nt1']/div")
        
                for div_content in div_contents:
                    walmart_product_detail[div_content.find_element(By.TAG_NAME, 'h3').text.strip()] = div_content.find_element(By.TAG_NAME, "span").text.strip()

        
        return walmart_product_detail

    def quit(self):
        self.driver.quit()

In [6]:
def get_all_contexts(text, target_word, context_size=5):
    # Tokenize the text
    tokens = word_tokenize(text)

    tokens = [token for token in tokens if token not in string.punctuation]
    # Find all occurrences of the target word
    
    target_indices = [i for i, token in enumerate(tokens) if token.lower() == target_word.lower()]

    # Extract context sentences for each occurrence of the target word
    all_contexts = []
    for target_index in target_indices:
        start_index = max(0, target_index - context_size)
        end_index = min(len(tokens), target_index + context_size + 1)
        context_words = tokens[start_index:end_index]
        context_sentence = ' '.join(context_words)
        all_contexts.append(context_sentence)

    return all_contexts

In [7]:
def get_app_name(nlp, text):

    contexts = get_all_contexts(text, "app")

    print(contexts)
    app_name = defaultdict(int)

    for context in contexts:
        doc = nlp(context)

        #iterate through the entities
        for ent in doc.ents:
            name = re.sub("[^a-zA-Z0-9]", "", ent.text.upper())
            app_name[name]  +=1


    return max(app_name, key=app_name.get) if app_name else None

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/priyanshi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
#download punkt using nltk.download('punkt') and provide the path below
nltk.data.path.append("tokenizers")

#load the model
# nlp = spacy.load(r"C:\Users\moink\Downloads\white_label\model-best")
nlp = spacy.load(r"case_sensitive_ner")

/opt/homebrew/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [67]:
shared_folder_id = "1R9S8lc3N5nB0dqtIytS8g08K9Xn2NwFU"

#path to client secrets
# You need a Oauth2.0 Client to access the GoogleDrive, follow this link https://support.google.com/cloud/answer/6158849?hl=en 
#to create client id. Download the client secret and name the file as client_secrets.json, and stored in the same directory as this notebook or set client_config_file attr
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = "client_secrets.json"
drive = GoogleDriveWrapper()
folder_id = drive.createFolder("smart wifi plug".title(), shared_folder_id)
folder_id

'1MxfBZMJTHZFUOZn96I7kO61sibVVKyo7'

In [121]:
scraper = WalmartScraper ("smart wifi plug")

In [70]:
product_urls = scraper.getProductURLList()

In [71]:
len(product_urls)

1990

In [74]:
product_details= {}

In [ ]:

for product_url in product_urls:

    scraper.driver.get(product_url)
    product_name, product_id = scraper.getProductNameAndIDFromURL()

    print(product_name, product_id)
    if product_id not in product_details:
        product_details[product_id] = {'id': product_id}

        product_details[product_id]['name'] = product_name

        image_url = scraper.getLeftImageUrls()[0]
        product_detail = scraper.parseWalmartProduct()
        walmart_product_detail = scraper.parseWalmartProductDetails()


        product_details[product_id]['brand'] = product_detail['product_brand']
        product_details[product_id]['long_description'] = walmart_product_detail['long_description']
        
        description_list = [walmart_product_detail['long_description'], walmart_product_detail['short_description']]

        product_details[product_id]['app_name'] = None
        for description in description_list:
            if description:
                product_details[product_id]['app_name'] = get_app_name(nlp, description.replace("\n", " "))
                if product_details[product_id]['app_name']:
                    break

        print(product_details[product_id]['app_name'])

        image_name = f"{product_id}_{product_name.replace(' ', '-')}.jpg"
        #print(image['image_url'])
        
        #check if file already present
        file_id = drive.getFileID(file_name=image_name, parent_id=folder_id)
    
        #file not present
        if not file_id:
            #upload file under the parent folder with folder id
            file_id = drive.uploadFile(file_name=image_name, file_url= image_url, parent_id=folder_id)


        product_details[product_id]['image_url'] = image_url
        product_details[product_id]['url'] = product_url
        time.sleep(random.randrange(1,5))

In [147]:
len(product_details)

1137

In [149]:
df = pd.DataFrame(product_details.values())
# df = df.loc[:, df.columns != 'images']
df = df.dropna(subset=['url'])
df.to_csv("walmart_products_dataset.csv", index=False, encoding='utf-8')